In [1]:
from pymongo import MongoClient
import json
import re
import pandas as pd

In [2]:
client = MongoClient('localhost', 27017)

In [3]:
db = client['Project2']

# Steam Cleaning

In [4]:
games = db.Steam

In [5]:
# count documents
games.count_documents({})

5209

In [4]:
# build dictionary for roman numbers
# We romove "i" as it is often used as in english it means "io"
trans = {'ii':2, 'iii': 3, 'iv': 4, 'v': 5, 'vi':6, 'vii':7, 'viii':8, 'ix':9, 'x': 10,
         'xi': 11, 'xii': 12, 'xiii': 13, 'xiv':14, 'xv':15, 'xvi':16, 'xvii':17, 'xviii':18, 'xix':19, 'xx':20,
        'xxi':21, 'xxii':22, 'xxiii':23, 'xxiv':24, 'xxv':25, 'xxvi':26, 'xxvii':27, 'xxviii':28, 'xxix':29, 'xxx':30}

In [7]:
for game in games.find():
    # add an attribute "normTitle" where the title is set to lowercase and is without punctuation
    normTitle = game['title']
    # remove punctuation and turn to lowercase
    normTitle = re.sub(r'[^\w\s]', "" , normTitle.lower())
    # remove "_" from title. eg: "watch_dogs"
    normTitle = re.sub("_", " ", normTitle)
    # remove "ea sports"
    normTitle = re.sub("ea sports ", "", normTitle)
    # remove all double spaces
    normTitle = re.sub("  ", " ", normTitle)
    # make sure there are no spaces at the beggining or the end of the title
    normTitle = normTitle.strip()
    # create an array of the words contained in the title
    splitTitle = normTitle.split(" ")
    # loop through the world of the title
    for j in range(len(splitTitle)):
        word = splitTitle[j]
        # if the word is a roman number, turn it to arabic
        if word in trans:
            normTitle = (' '.join(splitTitle[:j]) + ' ' + str(trans[word]) + ' ' + ' '.join(splitTitle[j+1:])).strip()
    games.update_one({'_id': game['_id']}, {'$set' : {'normTitle': normTitle}})

In [9]:
# Normalize release date
for game in games.find():
    date = game['relDate']
    today = pd.to_datetime('2022-12-20')
    date = pd.to_datetime(date)
    # eliminate records where  the release date is after today (not yet released)
    if date > today:
        games.delete_one({'_id': game['_id']})
    else:
        try:
            date = date.strftime('%Y-%m-%d')
            games.update_one({'_id': game['_id']}, {'$set' : {'relDate': date}})
        except:
            continue

In [ ]:
# normalize prices
for game in games.find():
    price = game['price']
    # If the game is Free to Play, set the price to 0
    if price in ["Free-to-Play", "Free To Play", "Free to Play", "Gratis", ""]:
        price = "0"
    # Remove the € symbol
    if price[-1] == "€":
        price = price[:-1]
    # Some prices have two "--" after the point (eg: 13.--). Remove them.
    if price[-1] == "-":
        price = price[:-3]
    # Update the price
    games.update_one({'_id': game['_id']}, {'$set' : {'price': price}})

In [7]:
# turn to dataframe
data = pd.json_normalize(games.find())

In [8]:
# remove duplicates based on SN
games.delete_many( { '_id': {'$in': list(data[data.duplicated('SN')]['_id'])}})

In [13]:
# remove all games that have the attribute "todrop"=True
games.delete_many({'toDrop': True})

In [14]:
# remove games with "coming soon" release date
games.delete_many({'relDate': 'Coming soon'})

In [15]:
# count documents
games.count_documents({})

5243

In [16]:
# update dataframe
data = pd.json_normalize(games.find())

In [17]:
# remove all games with "bundle" inside the title
# bundle: a collection of different games
for game in games.find():
    if "bundle" in game['normTitle']:
        games.delete_one({'_id': game['_id']})

In [18]:
# count documents
games.count_documents({})

5227

In [19]:
# create a second normTitle without edition and expansion keywords
for game in games.find():
    normTitle2 = game['normTitle']
    normTitle2 = re.sub("deluxe edition", "", normTitle2)
    normTitle2 = re.sub("enhanced edition", "", normTitle2)
    normTitle2 = re.sub("definitive edition", "", normTitle2)
    normTitle2 = re.sub("complete edition", "", normTitle2)
    normTitle2 = re.sub("gold edition", "", normTitle2)
    normTitle2 = re.sub("game of the year edition", "", normTitle2)
    normTitle2 = re.sub("goty edition", "", normTitle2)
    normTitle2 = re.sub("season pass edition", "", normTitle2)
    normTitle2 = re.sub("special edition", "", normTitle2)
    normTitle2 = re.sub("anniversary edition", "", normTitle2)
    normTitle2 = re.sub("expansion", "", normTitle2)
    normTitle2 = re.sub("pack", "", normTitle2)
    # remove double spaces and spaces at the beginning and at the end of the title
    normTitle2 = re.sub("  ", " ", normTitle2)
    normTitle2 = normTitle2.strip()
    # if the title contains this words
    if normTitle2 != game['normTitle']:
        # if the title without the keywords already exists we delete it
        if normTitle2 in list(data['normTitle']):
            games.delete_one({'_id': game['_id']})
        # if it doesn't, we save normTitle2 to the document, as it can be useful when merging
        else:
            games.update_one({'_id': game['_id']}, {'$set' : {'normTitle2': normTitle2}})

In [20]:
games.count_documents({})

5209

# MetacriticPC Cleaning

In [9]:
reviews = db.metacriticPc

In [10]:
reviews.count_documents({})

78456

### Remove duplicates

In [17]:
# turn to dataframe
data = pd.json_normalize(reviews.find())

In [18]:
# group by title
group = data.groupby('normTitle').count()

In [19]:
# get the titles of the games that appear more than once
dupl = list(group[group['_id'] > 1].index)

In [20]:
# there are 89 duplicates
len(dupl)

320

In [21]:
# remove the first of the two duplicate
# We do this because the two records are always equal except for the release date. We want to keep the oldest of the two.
for el in dupl:
    objId = list(data[data['normTitle'] == el]['_id'])[:-1]
    for i in objId:
        reviews.delete_one({'_id': i})

In [22]:
reviews.count_documents({})

78130

In [45]:
# add normTitle (same code as above)
for review in reviews.find():
    normTitle = review['title']
    normTitle = re.sub(r'[^\w\s]', "" , normTitle.lower())
    normTitle = re.sub("_", " ", normTitle)
    normTitle = re.sub("ea sports ", "", normTitle)
    normTitle = re.sub("  ", " ", normTitle)
    normTitle = normTitle.strip()
    splitTitle = normTitle.split(" ")
    for j in range(len(splitTitle)):
        word = splitTitle[j]
        if word in trans:
            normTitle = (' '.join(splitTitle[:j]) + ' ' + str(trans[word]) + ' ' + ' '.join(splitTitle[j+1:])).strip()
    reviews.update_one({'_id': review['_id']}, {'$set' : {'normTitle': normTitle}})

In [46]:
# Normalize release date
for review in reviews.find():
    date = review['relDate']
    date = pd.to_datetime(date)
    try:
        date = date.strftime('%Y-%m-%d')
        reviews.update_one({'_id': review['_id']}, {'$set' : {'relDate': date}})
    except:
        continue

# MetacriticALL Cleaning

In [23]:
reviews = db.metacriticAll

In [24]:
reviews.count_documents({})

26662

As the page with all the review contains reviews of the same game in different platforms, we are going to use as proxy of the pc review of a game the mean of the review of the same game in all the others platforms.
To do this, we group by title and calculate the review mean.

For what regards the release date, instead, we use the min of all the release dates.

In [25]:
# turn to dataframe
data = pd.json_normalize(reviews.find())

In [52]:
# create a subset with only the games that have a score
dataSub = data[data['score'] != 'tbd']

In [53]:
# turn the score column to integer
dataSub = dataSub.astype({'score':'int'})

In [54]:
# convert relDate column to datetime object
dataSub['relDate'] = pd.to_datetime(dataSub['relDate'])

In [55]:
# groupby title and calculate the mean of the scores
data1 = dataSub.groupby('title').mean('score')

In [56]:
# groupby title and calculate the min of the relDate
data2 = dataSub.groupby('title').min()['relDate']

In [57]:
# merge the 2 dataframes togheter
dataSub = pd.merge(data1, data2, on='title')

In [58]:
# bring back the title to the columns
dataSub['title'] = dataSub.index
# reorder the columns
dataSub = dataSub[['title', 'score', 'relDate']]

In [59]:
gameWScore = list(dataSub['title'])

In [60]:
# drop a game without a score, if it is already present in the subset with score
dataSub2 = []
for game in reviews.find():
    if game['score'] != 'tbd':
        continue
    if game['title'] in gameWScore:
        continue
    dataSub2.append(game)

In [61]:
# turn to dataframe
dataSub2 = pd.json_normalize(dataSub2)

In [62]:
# convert relDate column to datetime object
dataSub2['relDate'] = pd.to_datetime(dataSub2['relDate'])

In [63]:
# eliminate the duplicates inside the second subset
dataSub2 = pd.DataFrame(dataSub2.groupby('title').min()['relDate'])

In [64]:
dataSub2['title'] = dataSub2.index
dataSub2['score'] = 'tbd'
dataSub2 = dataSub2[['title', 'score', 'relDate']]

In [66]:
data = pd.concat([dataSub, dataSub2])

In [70]:
# normalize the date column
data['relDate'] = data['relDate'].dt.strftime('%Y-%m-%d')

In [71]:
# turn back into a dictionary
newReviews = data.to_dict(orient='records')

In [72]:
# add normTitle
for review in newReviews:
    normTitle = review['title']
    normTitle = re.sub(r'[^\w\s]', "" , normTitle.lower())
    normTitle = re.sub("_", " ", normTitle)
    normTitle = re.sub("ea sports ", "", normTitle)
    normTitle = re.sub("  ", " ", normTitle)
    normTitle = normTitle.strip()
    splitTitle = normTitle.split(" ")
    for j in range(len(splitTitle)):
        word = splitTitle[j]
        if word in trans:
            normTitle = (' '.join(splitTitle[:j]) + ' ' + str(trans[word]) + ' ' + ' '.join(splitTitle[j+1:])).strip()
    review['normTitle'] = normTitle

In [ ]:
# turn to dataframe
data = pd.json_normalize(newReviews)

In [30]:
# remove duplicates based on normTitle
data = data[~data.duplicated(subset='normTitle', keep='last')]

In [31]:
# turn back into a dictionary
newReviews = data.to_dict(orient='records')

In [34]:
# drop the entire collection
reviews.drop()

In [35]:
# and replace it with the cleaned one
metaAll = db.metacriticAll
metaAll.insert_many(newReviews)